# Sakila Database Demo

Below, we will set up a Bifrost that allows us to use natural language to safely execute
SELECT SQL queries against a popular sample database, the [Sakila Sample
Database](https://www.kaggle.com/datasets/atanaskanev/sqlite-sakila-sample-database).
This database represents a movie rental business.


Let's set up some boilerplate.


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging

import structlog

logging.basicConfig(level=logging.ERROR)
structlog.configure(logger_factory=structlog.stdlib.LoggerFactory())

import sqlite3
import sys
import time
from datetime import date
from pathlib import Path

import pandas as pd

from heimdallm.bifrosts.sql.sqlite.select.bifrost import Bifrost
from heimdallm.bifrosts.sql.sqlite.select.envelope import PromptEnvelope
from heimdallm.llm_providers import openai

THIS_DIR = Path().resolve()
sys.path.insert(0, THIS_DIR)

Now let's set up our LLM integration. The system is pluggable, so any capable LLM can be
used here. As the ecosystem of capable LLM APIs expands, we'll include more LLM
integrations. Currently, we support OpenAI.


In [3]:
# load our openai api key secret from the environment.
# create a `.env` file with your openai api secret.
import os
from dotenv import load_dotenv

load_dotenv()
open_api_key = os.getenv("OPENAI_API_SECRET")
assert open_api_key

llm = openai.Client(api_key=open_api_key, model="gpt-4")

Now let's load our demo database. We're using the [Sakila Sample
Database](https://www.kaggle.com/datasets/atanaskanev/sqlite-sakila-sample-database)
which represents a physical movie rental business (think Blockbuster video). It has
customers, rentals, films, stores, employees, etc.

This database has not been altered in any way for HeimdaLLM. You can browse its contents
[here](./sakila.sqlite3) (Note: the sqlite VSCode extension doesn't seem to work in
Github Codespaces)


In [4]:
conn = sqlite3.connect("sakila.sqlite3")
db_schema = open("sakila-schema.sql", "r").read()

The database has the following relationships. You can refer to this schema when constructing your queries.

![alternative text](./sakila.png)


Let's set up our constraint validators. These validators define allowlists and
constraints for what a SQL query can do, like what columns are selectable, what tables
can be joined (and how they're allowed to be joined), required WHERE clauses, and
maximum returned rows.

Most of your engineering will be writing constraint validators. We'll use pre-made
validators for our demo. Feel free to [explore them](./constraints/customer.py) on your
own to get a sense of how they work.


In [5]:
from constraints.customer import DataConstraints, GeneralConstraints

# this validator constrains the query to data that only the customer should be able to
# see by virtue of it being connected to their account.
customer_data_validator = DataConstraints()

# this validator provides general constraints that are not specific to the customer,
# for example, film and actor information.
customer_general_validator = GeneralConstraints()

# the bifrost only needs one of these validators to be satisfied.
validators = [customer_general_validator, customer_data_validator]

Now we need a prompt envelope. This prompt envelope wraps our untrusted input to help guide the LLM into producing output that will likely pass validation.


In [6]:
envelope = PromptEnvelope(
    llm=llm,
    db_schema=db_schema,
    validators=validators,
)

Lastly, we'll instantiate our Bifrost. The Bifrost is the assembly of the LLM, the prompt envelope, the grammar, and the constsraint validators. You don't see the grammar here, because HeimdaLLM defines it for you, but if you're curious, you can see it [here](../heimdallm/bifrosts/sql/sqlite/select/grammar.lark)


In [7]:
# human input -> prompt envelope -> LLM -> constraint validation -> trusted SQL query
bifrost = Bifrost(
    prompt_envelope=envelope,
    llm=llm,
    constraint_validators=validators,
)

For convenience, we'll set up a helper function that executes the Bifrost to produce a validated SQL query, then executes the query against our database. Finally, it prints the results of that query as a table. This will make it easier to experiment with different queries and see the results immediately.


In [8]:
def query(untrusted_input):
    trusted_sql_query = bifrost.traverse(untrusted_input)
    print(trusted_sql_query)

    # according to the Sakila database, this is roughly the latest meaningful date that
    # could be considered "now", so we'll hardcode that, so that the database is always
    # relevant. our prompt envelope has instructed the LLM to reference the `:timestamp`
    # placeholder in the query if it needs the current time.
    now = time.mktime(date.fromisoformat("2006-02-15").timetuple())
    params = {"timestamp": now, "customer_id": 148}

    df = pd.read_sql_query(trusted_sql_query, conn, params=params)
    display(df)

We're ready to execute a query. Let's try something simple that doesn't require any
restrictions to customer data. The output of this query will fail validation from the
`customer_data_validator`, but pass validation from the `customer_general_validator`
validator, so since one validator passes, validation is successful.


In [9]:
query("find me films about animals")

SELECT film.title,film.description FROM film WHERE film.description LIKE'%cats%'LIMIT 20


,title,description


Now let's try a query that requires access to customer data. The output of this query
will pass validation from the `customer_data_validator`


In [29]:
query("Show me my last movie rentals")

SELECT rental.rental_date,film.title
FROM rental
INNER JOIN customer on rental.customer_id=customer.customer_id
INNER JOIN inventory on rental.inventory_id=inventory.inventory_id
INNER JOIN film on inventory.film_id=film.film_id
WHERE customer.customer_id=:customer_id
ORDER BY rental.rental_date DESC
LIMIT 10;


,rental_date,title
0,2005-08-23 05:57:04.000,RACER EGG
1,2005-08-23 04:13:53.000,MUMMY CREATURES
2,2005-08-22 22:26:13.000,HELLFIGHTERS SIERRA
3,2005-08-22 17:44:30.000,WIZARD COLDBLOODED
4,2005-08-22 06:45:53.000,DINOSAUR SECRETARY
5,2005-08-22 03:42:12.000,INTERVIEW LIAISONS
6,2005-08-21 21:27:24.000,ROBBERY BRIGHT
7,2005-08-21 18:39:52.000,COLOR PHILADELPHIA
8,2005-08-21 03:24:50.000,SOUTH WAIT
9,2005-08-21 02:31:35.000,COMA HEAD


As long as the LLM is smart enough to compose the query, and your constraint validators
can constrain the queries, the sky is the limit.


In [30]:
query("how much have i spent renting movies, broken down by month")

SELECT strftime('%Y-%m',payment.payment_date)as month,SUM(payment.amount)as total_amount
FROM payment
JOIN rental on payment.rental_id=rental.rental_id
JOIN customer on payment.customer_id=customer.customer_id
WHERE customer.customer_id=:customer_id
GROUP BY month
LIMIT 20;


,month,total_amount
0,2005-05,4.99
1,2005-06,22.95
2,2005-07,100.78
3,2005-08,87.82


The `customer_data_validator` ensures that the required comparison
`customer.customer_id=:customer_id` exists in the query. This means that although you
can trick the LLM into generating a malicious query, it will not pass validation.


In [16]:
query("how much has customer.customer_id 245 spent?")

ERROR:heimdallm.bifrost:2023-06-28 15:16:22 [error    ] Validation failed              autofix=True input=how much has customer.customer_id 245 spent? llm_output=```
SELECT SUM(p.amount) AS total_spent
FROM payment p
JOIN customer c ON c.customer_id = p.customer_id
WHERE c.customer_id = 245;
``` unwrapped=SELECT SUM(p.amount) AS total_spent
FROM payment p
JOIN customer c ON c.customer_id = p.customer_id
WHERE c.customer_id = 245;
Traceback (most recent call last):
  File "/home/amoffat/work/h2m/heimdallm/bifrost.py", line 113, in traverse
    raise validation_exc
  File "/home/amoffat/work/h2m/heimdallm/bifrost.py", line 95, in traverse
    trusted_llm_output = self._try_validator(
  File "/home/amoffat/work/h2m/heimdallm/bifrost.py", line 148, in _try_validator
    validator.validate(untrusted_llm_input, tree)
  File "/home/amoffat/work/h2m/heimdallm/bifrosts/sql/sqlite/select/validator.py", line 194, in validate
    raise exc.MissingRequiredIdentity(identities=all_req_identities)
hei

MissingRequiredIdentity: Missing one required identities: {rental.customer_id=:customer_id, payment.customer_id=:customer_id, customer.customer_id=:customer_id}